In [ ]:
from google.colab import drive
drive.mount("/data")

Mounted at /data


In [ ]:
cd /data/MyDrive/Dataset/data

/data/MyDrive/Dataset/data


In [ ]:
import numpy as np
import pandas as pd

import json

from tqdm.notebook import tqdm

In [ ]:
empties = pd.read_csv("<path_to_labeled_data_with_no_bundles>")
print(empties.shape)
empties.head()

(808, 5)


,Text,WallId,PostId,Source,GptResponse
0,"клининг/фея чистоты/ уборка 🧚\nофис, квартира,...",1121942433,400700,tg,NaN
1,Распродажа \nЦена: 550р шт \nРазмер: 41.42.43....,-206817857,82815,vk,NaN
2,katrom12 нарушил правила чата и не сможет писа...,1332882905,691243,tg,NaN
3,"добрый день. \nкуплю машину, модели с большим ...",1121942433,223118,tg,NaN
4,"чехол для айфоновских наушников 1,2 за вкусняш...",1121942433,427581,tg,NaN


In [ ]:
data = pd.read_csv("<path_to_labeled_and_validated_data>").drop(columns="Unnamed: 0")
print(data.shape)
data.head()

(19214, 10)


,Text,WallId,PostId,Source,Title,Price,Currency,Count,Label_gpt,Comment_gpt
0,"1. монитор, 250 лари\n2. мольберт, 50 лари\n3....",1332882905,686748,tg,"рюкзак, 40 литров, идеальное состояние",100,лари,1,valid,The offer includes a backpack bundle that matc...
1,#БНИ_ПродамБУ_Москва\n#БНИ_ПродамНовая_Москва\...,-114967596,592027,vk,"Street Fighter (база, стретчи, доп)",26000,RUB,1,valid,The offer clearly describes the bundle's conte...
2,Продам стульчик для кормления Happy baby. В хо...,-82544098,661957,vk,Стульчик для кормления Happy baby,3000,RUB,1,valid,The offer clearly describes a single feeding c...
3,"Продаю компьютер Цена 55000 Все почти новое ,е...",-90442795,353383,vk,Компьютер с комплектующими и аксессуарами,55000,RUB,1,valid,The offer clearly describes a computer with co...
4,Новинка сезона \nКачество очень хорошее \nЦена...,-192083416,15466,vk,Новинка сезона,2000,RUB,1,invalid,"The OFFER lacks detailed product information, ..."


In [ ]:
data = data[data["Label_gpt"] == "valid"]

In [ ]:
data.shape, data["Text"].nunique()

((16098, 10), 9903)

## Приведём к виду для обучения

In [ ]:
data = data.drop(columns=["WallId", "PostId", "Source", "Label_gpt", "Comment_gpt"])
data.head()

,Text,Title,Price,Currency,Count
0,"1. монитор, 250 лари\n2. мольберт, 50 лари\n3....","рюкзак, 40 литров, идеальное состояние",100,лари,1
1,#БНИ_ПродамБУ_Москва\n#БНИ_ПродамНовая_Москва\...,"Street Fighter (база, стретчи, доп)",26000,RUB,1
2,Продам стульчик для кормления Happy baby. В хо...,Стульчик для кормления Happy baby,3000,RUB,1
3,"Продаю компьютер Цена 55000 Все почти новое ,е...",Компьютер с комплектующими и аксессуарами,55000,RUB,1
5,"продам часы galaxy watch\nпокупали на амазоне,...",часы galaxy watch с стальным ремешком,200,EUR,1


In [ ]:
col2id = dict(zip(data.columns, np.arange(len(data.columns))))
col2id

{'Text': 0, 'Title': 1, 'Price': 2, 'Currency': 3, 'Count': 4}

In [ ]:
def make_json(row):
    columns = ("Title", "Price", "Currency", "Count")
    return json.dumps({col: row[col2id[col]] for col in columns})


data["bundle"] = data.apply(make_json, axis=1)
data.drop(columns=["Title", "Price", "Currency", "Count"], inplace=True)
data.head()

,Text,bundle
0,"1. монитор, 250 лари\n2. мольберт, 50 лари\n3....","{""Title"": ""\u0440\u044e\u043a\u0437\u0430\u043..."
1,#БНИ_ПродамБУ_Москва\n#БНИ_ПродамНовая_Москва\...,"{""Title"": ""Street Fighter (\u0431\u0430\u0437\..."
2,Продам стульчик для кормления Happy baby. В хо...,"{""Title"": ""\u0421\u0442\u0443\u043b\u044c\u044..."
3,"Продаю компьютер Цена 55000 Все почти новое ,е...","{""Title"": ""\u041a\u043e\u043c\u043f\u044c\u044..."
5,"продам часы galaxy watch\nпокупали на амазоне,...","{""Title"": ""\u0447\u0430\u0441\u044b galaxy wat..."


In [ ]:
def aggregate_json(group):
    bundles = []
    for ind in group.index:
        bundles.append(json.loads(group.loc[ind]))
    return json.dumps(bundles)


data = data.groupby("Text", as_index=False).agg(bundles=("bundle", aggregate_json), n_bundles=("bundle", 'count'))
data.set_index(np.arange(len(data)), inplace=True)
data.head()

,Text,bundles,n_bundles
0,! новая. хлопковая рубашка оверсайз с приятной...,"[{""Title"": ""\u0445\u043b\u043e\u043f\u043a\u04...",1
1,!!! ПОМИДОРКИ !!! в 3л Банке.... по 250 руб. К...,"[{""Title"": ""\u0418\u043a\u0440\u0430 \u041a\u0...",3
2,"""конфетный мячик"" 🍬🏀 цена €7😻 бесплатная доста...","[{""Title"": ""\u043a\u043e\u043d\u0444\u0435\u04...",1
3,# ТРИКО Распродажа без выбораМОДЕЛЬ ТКАНЬ ТРИК...,"[{""Title"": ""\u0422\u0440\u0438\u043a\u043e, \u...",1
4,# ТРИКО Распродажа без выбораМОДЕЛЬ ТКАНЬ ТРИК...,"[{""Title"": ""\u0421\u0443\u043f\u0435\u0440\u04...",1


In [ ]:
data.shape

(9903, 3)

In [ ]:
empties.drop(columns=["WallId", "PostId", "Source", "GptResponse"], inplace=True)
empties["bundles"] = json.dumps([])
empties["n_bundles"] = 0

empties.head()

,Text,bundles,n_bundles
0,"клининг/фея чистоты/ уборка 🧚\nофис, квартира,...",[],0
1,Распродажа \nЦена: 550р шт \nРазмер: 41.42.43....,[],0
2,katrom12 нарушил правила чата и не сможет писа...,[],0
3,"добрый день. \nкуплю машину, модели с большим ...",[],0
4,"чехол для айфоновских наушников 1,2 за вкусняш...",[],0


In [ ]:
data = pd.concat([data, empties]).sample(frac=1)
data.head()

,Text,bundles,n_bundles
358,#продаю в тбилиси focusrite scatlett 2i2. полн...,"[{""Title"": ""focusrite scatlett 2i2"", ""Price"": ...",1
5640,здравствуйте продам набор для наращивания ресн...,"[{""Title"": ""\u041d\u0430\u0431\u043e\u0440 \u0...",1
8501,продаю оригинальные мужские кожаные ремни:\n\n...,"[{""Title"": ""\u043c\u0443\u0436\u0441\u043a\u04...",2
3494,"Продаю билет на аэроэкспресс, срок действия до...","[{""Title"": ""\u0411\u0438\u043b\u0435\u0442 \u0...",1
6233,масло для тела с алоэ. новое. 2€ бутылочка. по...,"[{""Title"": ""\u043c\u0430\u0441\u043b\u043e \u0...",1


In [ ]:
data["n_bundles"].value_counts()

n_bundles
1     7905
0      808
2      728
3      399
4      279
5      149
6      132
7       95
10      70
9       55
8       54
13       6
12       6
11       6
17       4
18       4
15       3
16       2
35       1
36       1
28       1
22       1
14       1
27       1
Name: count, dtype: int64

In [ ]:
data.to_csv("<path_to_final_dataset>", index=False)